In [1]:
# !pip install thefuzz
# !pip install pandas fuzzywuzzy python-Levenshtein

In [2]:
import os
import pandas as pd
import numpy as np
from fuzzywuzzy import process
from thefuzz import process, fuzz
import utils
import time

import warnings
warnings.filterwarnings('ignore')

## Data Loading and Preliminary Inspection

In [3]:
df_Compustat = pd.read_csv('Compustat for ODI.csv')
df_ODI = pd.read_csv('ODI 2002-2011.csv') # save with encoding (UTF-8) in sublime before using 

#### Compustat for ODI.csv

In [4]:
df_Compustat.shape

(113332, 981)

In [5]:
obj_cols = df_Compustat.select_dtypes('object').columns.tolist()
obj_cols
# df_Compustat[obj_cols]

['datadate',
 'indfmt',
 'consol',
 'popsrc',
 'datafmt',
 'tic',
 'cusip',
 'conm',
 'acctchg',
 'acctstd',
 'acqmeth',
 'bspr',
 'compst',
 'curcd',
 'curncd',
 'final',
 'ogm',
 'stalt',
 'apdedate',
 'fdate',
 'pdate',
 'costat',
 'fic',
 'ceoso',
 'cfoso',
 'add1',
 'add2',
 'add3',
 'add4',
 'addzip',
 'busdesc',
 'city',
 'conml',
 'county',
 'ein',
 'fax',
 'idbflag',
 'incorp',
 'loc',
 'phone',
 'prican',
 'prirow',
 'spcsrc',
 'state',
 'weburl',
 'dldte',
 'ipodate']

In [6]:
df_Compustat.head()
# df_Compustat.columns

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,...,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate
0,1004,2001-05-31,2000,INDL,C,D,STD,AIR,000361105,AAR CORP,...,1.0,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,1972-04-24
1,1004,2002-05-31,2001,INDL,C,D,STD,AIR,000361105,AAR CORP,...,1.0,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,1972-04-24
2,1004,2003-05-31,2002,INDL,C,D,STD,AIR,000361105,AAR CORP,...,1.0,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,1972-04-24
3,1004,2004-05-31,2003,INDL,C,D,STD,AIR,000361105,AAR CORP,...,1.0,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,1972-04-24
4,1004,2005-05-31,2004,INDL,C,D,STD,AIR,000361105,AAR CORP,...,1.0,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,1972-04-24


#### ODI 2002-2011.csv

In [7]:
df_ODI.shape

(649925, 34)

In [8]:
df_ODI.head()

,STATUS,ESTAB_NAME,ESTAB_NAME2,STREET,CITY,STATE,ZIP,Year,EMP_Q1,HOURS_Q2,...,PHONE,UNUSUAL_Q3,STRIKE_Q3,SHUT_Q3,SEASONAL_Q3,DISASTER_Q3,SHORT_Q3,LONG_Q3,OREASON_Q3,OREASON_DESC
0,OK,PUBLISHERS PRINTING CO LLC,NaN,13487 S PRESTON HWY,LEBANON JUNCTION,KY,40150,2005,838.0,1743040.0,...,5025432251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,temp layoff due 2 lack of wk less than 40 empl...
1,OK,Publishers Printing Co LLC,NaN,100 Frank E Simon Ave,Shepherdsville,KY,40165,2005,872.0,1813760.0,...,5025432251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Temp layoff due2 lack of work 4 less than 40 p...
2,OK,Bonnie Brae Convalescent Hosp,Bonnie Brae,420 S Bonnie Brae St,Los Angeles,CA,90057,2005,52.0,88000.0,...,2134838144,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,OK,Colwell Construction Co Inc,NaN,1208 Rock Rd,Blairsville,GA,30512,2005,45.0,10000.0,...,7067456239,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN
4,OK,Te Slaa Trucking Ltd,Teslaa Trucking,3111 320th St,Hull,IA,51239,2005,41.0,100862.0,...,7124391382,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


#### Select part of the data as the test set

In [9]:
df1 = df_Compustat.head(15000)
df2 = df_ODI.head(15000)

In [10]:
df1.to_csv('Comp_test.csv')
df2.to_csv('ODI_test.csv')

## Fuzzy Text Matching for Data Merging

### Normalization
#### Convert all the letters to lower case

In [11]:
df_Compustat_ = df_Compustat.apply(lambda x: x.str.lower() if x.dtype == "object" else x)
df1_ = df1.apply(lambda x: x.str.lower() if x.dtype == "object" else x)


In [12]:
df_ODI_ = df_ODI.apply(lambda x: x.str.lower() if x.dtype == "object" else x)
df2_ = df2.apply(lambda x: x.str.lower() if x.dtype == "object" else x)
# df2

In [13]:
# sort ODI by names 
df_ODI_sorted = df_ODI.sort_values('ESTAB_NAME')
df_ODI_sorted.head(5)

,STATUS,ESTAB_NAME,ESTAB_NAME2,STREET,CITY,STATE,ZIP,Year,EMP_Q1,HOURS_Q2,...,PHONE,UNUSUAL_Q3,STRIKE_Q3,SHUT_Q3,SEASONAL_Q3,DISASTER_Q3,SHORT_Q3,LONG_Q3,OREASON_Q3,OREASON_DESC
183063,OK,"""K"" Line America, Inc.",NaN,8730 Stony Point Pkwy,Richmond,VA,23235,2004,485.0,235417.0,...,8045603600,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
133566,OK,"#1 Steele, Inc.",River Oaks Nursing Home,1001 N Walnut St,Steele,MO,63877,2004,45.0,62600.0,...,5736952121,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
398857,OK,% U-Haul Company of Ma.,U-Haul MCO Of Boston (837-069),151 Linwood St,Somerville,MA,02143,2008,37.0,64885.0,...,8005280361,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
81173,OK,% U-Haul Company of Ma.,U-Haul MCO Of Boston (837-069),151 Linwood St,Somerville,MA,02143,2003,12.0,25162.0,...,8005280361,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
361726,OK,% U-Haul Company of Ma.,U-Haul MCO Of Boston (837-069),151 Linwood St,Somerville,MA,02143,2007,40.0,78000.0,...,8005280361,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [ ]:
# sort Comp by names
df_Compustat_sorted = df_Compustat.sort_values('conm')
df_Compustat_sorted.head(5)

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,...,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate
63926,66425,2005-12-31,2005,INDL,C,D,STD,CTAC.1,681977104,1-800 CONTACTS INC,...,1.0,5961,447.0,976.0,C,UT,0.0,www.1800contacts.com,2007-09-07,1998-02-09
63925,66425,2004-12-31,2004,INDL,C,D,STD,CTAC.1,681977104,1-800 CONTACTS INC,...,1.0,5961,447.0,976.0,C,UT,0.0,www.1800contacts.com,2007-09-07,1998-02-09
63927,66425,2006-12-31,2006,INDL,C,D,STD,CTAC.1,681977104,1-800 CONTACTS INC,...,1.0,5961,447.0,976.0,C,UT,0.0,www.1800contacts.com,2007-09-07,1998-02-09
63924,66425,2003-12-31,2003,INDL,C,D,STD,CTAC.1,681977104,1-800 CONTACTS INC,...,1.0,5961,447.0,976.0,C,UT,0.0,www.1800contacts.com,2007-09-07,1998-02-09
63923,66425,2002-12-31,2002,INDL,C,D,STD,CTAC.1,681977104,1-800 CONTACTS INC,...,1.0,5961,447.0,976.0,C,UT,0.0,www.1800contacts.com,2007-09-07,1998-02-09


### Select relevant attributes for matching

In [ ]:
# df_Compustat_sorted[['conm', 'conml', 'add1', 'addzip', 'city', 'state', 'fyear']]
df1_ = df1_[['conm', 'conml', 'add1', 'addzip', 'city', 'state', 'fyear']]

In [ ]:
# df_ODI_sorted[['ESTAB_NAME', 'ESTAB_NAME2', 'STREET', 'ZIP', 'CITY', 'STATE', 'Year']]
df2_ = df2_[['ESTAB_NAME', 'ESTAB_NAME2', 'STREET', 'ZIP', 'CITY', 'STATE', 'Year']]

### Matching - one attribute
#### Fuzzy Text Matching

In [17]:
# Method 1
##### fuzzy match, add a new col indicating the matching score ##### 

def get_matches(query, choices, score, limit=3):
    results = process.extract(query, choices, scorer=fuzz.token_sort_ratio, limit=limit)
#     return results
    return [result for result in results if result[1] >= score]

# test
# Start the timer
start_time = time.time()

res = df1_.copy()
res['matched_NAME'] = res['conm'].apply(lambda x: get_matches(x, df2_['ESTAB_NAME'], 100))
fuzzy100_matches_1 = res[res['matched_NAME'].apply(lambda x: bool(x) and isinstance(x, list))]

print(f'Number of 100% fuzzy matches (Company Name): {len(fuzzy100_matches_1)}')

# End the timer
end_time = time.time()
duration = end_time - start_time
print(f"The code took {duration} seconds to execute.")


Number of 100% fuzzy matches (Company Name): 654
The code took 266.6744592189789 seconds to execute.


In [18]:
fuzzy100_matches_1.head() # updated df

,conm,conml,add1,addzip,city,state,fyear,matched_NAME
142,abbott laboratories,abbott laboratories,"100 abbott park road, abbott park",60064-6400,north chicago,il,2001,"[(abbott laboratories, 100, 12231)]"
143,abbott laboratories,abbott laboratories,"100 abbott park road, abbott park",60064-6400,north chicago,il,2002,"[(abbott laboratories, 100, 12231)]"
144,abbott laboratories,abbott laboratories,"100 abbott park road, abbott park",60064-6400,north chicago,il,2003,"[(abbott laboratories, 100, 12231)]"
145,abbott laboratories,abbott laboratories,"100 abbott park road, abbott park",60064-6400,north chicago,il,2004,"[(abbott laboratories, 100, 12231)]"
146,abbott laboratories,abbott laboratories,"100 abbott park road, abbott park",60064-6400,north chicago,il,2005,"[(abbott laboratories, 100, 12231)]"


In [19]:
# Method 2
##### fuzzy match with specific score, get the number of matches and the matched set #####

def fuzzy_matches(df1, df2, column1, column2, score):
    # Find matches where the fuzzy match score is 100
    matches = df1[column1].apply(lambda x: any(fuzz.ratio(x, y) >= score for y in df2[column2])) 
    
    return df1[matches]

# test
# Start the timer
start_time = time.time()

fuzzy100_matches_2 = fuzzy_matches(df1_, df2_, 'conm', 'ESTAB_NAME', 100)
print(f'Number of 100% fuzzy matches (Company Name): {len(fuzzy100_matches_2)}')
fuzzy100_matches_2.head()

# End the timer
end_time = time.time()
duration = end_time - start_time
print(f"The code took {duration} seconds to execute.")

Number of 100% fuzzy matches (Company Name): 530
The code took 136.10346794128418 seconds to execute.


In [20]:
fuzzy100_matches_2.head()

,conm,conml,add1,addzip,city,state,fyear
142,abbott laboratories,abbott laboratories,"100 abbott park road, abbott park",60064-6400,north chicago,il,2001
143,abbott laboratories,abbott laboratories,"100 abbott park road, abbott park",60064-6400,north chicago,il,2002
144,abbott laboratories,abbott laboratories,"100 abbott park road, abbott park",60064-6400,north chicago,il,2003
145,abbott laboratories,abbott laboratories,"100 abbott park road, abbott park",60064-6400,north chicago,il,2004
146,abbott laboratories,abbott laboratories,"100 abbott park road, abbott park",60064-6400,north chicago,il,2005


In [21]:
# Method 3
##### fuzzy match with score, new column which includes matching descriptions is added #####
##### The new column contains the matching info of another dataset (matched_value, fuzzy_score, index_in_df2) #####

def fuzzy_matches(df1, df2, column1, column2, score, col_name):
    # Initialize lists to store the matched values and scores
    matched_values = []
    matched_scores = []

    # Iterate over each row in df1
    for x in df1[column1]:
        best_match = None
        best_score = 0

        # Find the best match in df2
        for y in df2[column2]:
            current_score = fuzz.ratio(x, y)
            if current_score > best_score:
                best_score = current_score
                best_match = y

        # Check if the best score meets the threshold
        if best_score >= score:
            match_index = df2[df2[column2] == best_match].index[0]
            matched_values.append((best_match, best_score, match_index))
        else:
            matched_values.append(None)

    # Add the matched values and scores as new columns in df1
    df1[col_name] = matched_values
    
    return df1

# test
# Start the timer
start_time = time.time()

res = fuzzy_matches(df1_, df2_, 'conm', 'ESTAB_NAME', 100, 'matched_name')
fuzzy100_matches_3 = res[res['matched_name'].notnull()]
print(f'Number of 100% fuzzy matches (Company Name): {len(fuzzy100_matches_3)}')

# End the timer
end_time = time.time()
duration = end_time - start_time
print(f"The code took {duration} seconds to execute.")

Number of 100% fuzzy matches (Company Name): 530
The code took 130.509428024292 seconds to execute.


In [22]:
fuzzy100_matches_3.head()

,conm,conml,add1,addzip,city,state,fyear,matched_name
142,abbott laboratories,abbott laboratories,"100 abbott park road, abbott park",60064-6400,north chicago,il,2001,"(abbott laboratories, 100, 12231)"
143,abbott laboratories,abbott laboratories,"100 abbott park road, abbott park",60064-6400,north chicago,il,2002,"(abbott laboratories, 100, 12231)"
144,abbott laboratories,abbott laboratories,"100 abbott park road, abbott park",60064-6400,north chicago,il,2003,"(abbott laboratories, 100, 12231)"
145,abbott laboratories,abbott laboratories,"100 abbott park road, abbott park",60064-6400,north chicago,il,2004,"(abbott laboratories, 100, 12231)"
146,abbott laboratories,abbott laboratories,"100 abbott park road, abbott park",60064-6400,north chicago,il,2005,"(abbott laboratories, 100, 12231)"


In [23]:
df2_[df2_['ESTAB_NAME'] == 'abbott laboratories']

,ESTAB_NAME,ESTAB_NAME2,STREET,ZIP,CITY,STATE,Year
12231,abbott laboratories,ross products division,5309 village pkwy,72758,rogers,ar,2002


#### Basic String Matching

In [24]:
# Evaluating the result of 100% fuzzy match 
##### string matches #####

def string_matches(df1, df2, column1, column2):
    # Convert columns to sets for faster comparison
    set2 = set(df2[column2])
    # Find matches where the strings are exactly the same
    matches = df1[column1].apply(lambda x: x in set2)
    return df1[matches]

# test
# Start the timer
start_time = time.time()

string_matches = string_matches(df1_, df2_, 'conm', 'ESTAB_NAME')
print(f'Number of matches (Company Name): {len(string_matches)}')

# End the timer
end_time = time.time()
duration = end_time - start_time
print(f"The code took {duration} seconds to execute.")

Number of matches (Company Name): 530
The code took 0.007289886474609375 seconds to execute.


In [25]:
string_matches.head(20)

,conm,conml,add1,addzip,city,state,fyear,matched_name
142,abbott laboratories,abbott laboratories,"100 abbott park road, abbott park",60064-6400,north chicago,il,2001,"(abbott laboratories, 100, 12231)"
143,abbott laboratories,abbott laboratories,"100 abbott park road, abbott park",60064-6400,north chicago,il,2002,"(abbott laboratories, 100, 12231)"
144,abbott laboratories,abbott laboratories,"100 abbott park road, abbott park",60064-6400,north chicago,il,2003,"(abbott laboratories, 100, 12231)"
145,abbott laboratories,abbott laboratories,"100 abbott park road, abbott park",60064-6400,north chicago,il,2004,"(abbott laboratories, 100, 12231)"
146,abbott laboratories,abbott laboratories,"100 abbott park road, abbott park",60064-6400,north chicago,il,2005,"(abbott laboratories, 100, 12231)"
147,abbott laboratories,abbott laboratories,"100 abbott park road, abbott park",60064-6400,north chicago,il,2006,"(abbott laboratories, 100, 12231)"
148,abbott laboratories,abbott laboratories,"100 abbott park road, abbott park",60064-6400,north chicago,il,2007,"(abbott laboratories, 100, 12231)"
149,abbott laboratories,abbott laboratories,"100 abbott park road, abbott park",60064-6400,north chicago,il,2008,"(abbott laboratories, 100, 12231)"
150,abbott laboratories,abbott laboratories,"100 abbott park road, abbott park",60064-6400,north chicago,il,2009,"(abbott laboratories, 100, 12231)"
151,abbott laboratories,abbott laboratories,"100 abbott park road, abbott park",60064-6400,north chicago,il,2010,"(abbott laboratories, 100, 12231)"


In [26]:
df2_[df2_['ESTAB_NAME'] == 'honeywell international inc']

,ESTAB_NAME,ESTAB_NAME2,STREET,ZIP,CITY,STATE,Year
2492,honeywell international inc,honeywell,400 hickory dr,28754,mars hill,nc,2002
2544,honeywell international inc,honeywell,2275 newlands dr e,89408,fernley,nv,2002
4419,honeywell international inc,honeywell,117 e providencia ave,91502,burbank,ca,2002
5689,honeywell international inc,honeywell,750 e highway 77,38059,newbern,tn,2002
6157,honeywell international inc,honeywell,1401 w cypress creek rd,33309,fort lauderdale,fl,2002
6931,honeywell international inc,honeywell,111 s 34th st,85034,phoenix,az,2002
12078,honeywell international inc,honeywell,1515 w blancke st,07036,linden,nj,2002


Same results between basic matching method and 100% fuzzy matching

### Matching - Multiple pairs of attributes

Method 1 - Combined all the keys in both dfs. 

Method 2 - Merge fuzzy matched elements one by one.

In [27]:
# df1[['conm', 'conml', 'add1', 'addzip', 'city', 'state', 'fyear']]
# df2[['ESTAB_NAME', 'ESTAB_NAME2', 'STREET', 'ZIP', 'CITY', 'STATE', 'Year']]


In [28]:
def clean(df):
    # Remove columns that end with '_y'
    df = df[df.columns.drop(list(df.filter(regex='_y')))]

    # Rename columns to remove '_x'
    df = df.rename(columns=lambda x: x.replace('_x', ''))
    
    # Drop duplicates in dataframe
    return df.drop_duplicates()


##### Method 1 #####
def fuzzy_matches_combined(df1, df2, score, col_name):
    # Create a combined key in both dataframes (exclude YEAR)
    df1['combined'] = df1['conm'] + ',' + df1['add1'] + ',' + df1['addzip'] + ',' + df1['city'] + ',' + df1['state']
    df2['combined'] = df2['ESTAB_NAME'] + ',' + df2['STREET'] + ',' + df2['ZIP'] + ',' + df2['CITY'] + ',' + df2['STATE']

    matched_values = []

    # Iterate over each row in df1
    for x in df1['combined']:
        best_match = None
        best_score = 0

        # Find the best match in df2
        for y in df2['combined']:
            current_score = fuzz.ratio(x, y)
            if current_score > best_score:
                best_score = current_score
                best_match = y

        # Check if the best score meets the threshold
        if best_score >= score:
            match_index = df2[df2['combined'] == best_match].index[0]
            matched_values.append((best_match, best_score, match_index))
        else:
            matched_values.append(None)

    # Add the matched values as a new column in df1
    df1[col_name] = matched_values

    # Drop the combined key column
    df1.drop('combined', axis=1, inplace=True)
    df2.drop('combined', axis=1, inplace=True)

    return df1

In [29]:
##### Method 1 test #####
# Start the timer
start_time = time.time()

fuzzy_2 = fuzzy_matches_combined(df1_, df2_, 90, 'matched_value')
fuzzy_2 = fuzzy_2[fuzzy_2['matched_value'].notnull()]

# End the timer
end_time = time.time()
duration = end_time - start_time
print(f"The code took {duration} seconds to execute.")

fuzzy_2

The code took 166.95382618904114 seconds to execute.


,conm,conml,add1,addzip,city,state,fyear,matched_name,matched_value
1083,amgen inc,amgen inc,one amgen center drive,91320-1799,thousand oaks,ca,2001,"(amgen inc, 100, 1196)","(amgen inc,1 amgen center dr,91320-1799,thousa..."
1084,amgen inc,amgen inc,one amgen center drive,91320-1799,thousand oaks,ca,2002,"(amgen inc, 100, 1196)","(amgen inc,1 amgen center dr,91320-1799,thousa..."
1085,amgen inc,amgen inc,one amgen center drive,91320-1799,thousand oaks,ca,2003,"(amgen inc, 100, 1196)","(amgen inc,1 amgen center dr,91320-1799,thousa..."
1086,amgen inc,amgen inc,one amgen center drive,91320-1799,thousand oaks,ca,2004,"(amgen inc, 100, 1196)","(amgen inc,1 amgen center dr,91320-1799,thousa..."
1087,amgen inc,amgen inc,one amgen center drive,91320-1799,thousand oaks,ca,2005,"(amgen inc, 100, 1196)","(amgen inc,1 amgen center dr,91320-1799,thousa..."
...,...,...,...,...,...,...,...,...,...
11621,micron technology inc,micron technology inc.,8000 south federal way,83716-9632,boise,id,2008,"(micron technology inc, 100, 4980)","(micron technology inc,8000 s federal way,8371..."
11622,micron technology inc,micron technology inc.,8000 south federal way,83716-9632,boise,id,2009,"(micron technology inc, 100, 4980)","(micron technology inc,8000 s federal way,8371..."
11623,micron technology inc,micron technology inc.,8000 south federal way,83716-9632,boise,id,2010,"(micron technology inc, 100, 4980)","(micron technology inc,8000 s federal way,8371..."
11624,micron technology inc,micron technology inc.,8000 south federal way,83716-9632,boise,id,2011,"(micron technology inc, 100, 4980)","(micron technology inc,8000 s federal way,8371..."


In [30]:
##### Method 2 #####
##### Name & Add & Zip & City & State #####

def fuzzy_sep(df1_, df2_):
    fuzzy_name = fuzzy_matches(df1_, df2_, 'conm', 'ESTAB_NAME', 90, 'matched_name')
    fuzzy_add = fuzzy_matches(df1_, df2_, 'add1', 'STREET', 80, 'matched_address') 
    fuzzy_zip = fuzzy_matches(df1_, df2_, 'addzip', 'ZIP', 90, 'matched_zip')
    fuzzy_city = fuzzy_matches(df1_, df2_, 'city', 'CITY', 90, 'matched_city')
    fuzzy_state = fuzzy_matches(df1_, df2_, 'state', 'STATE', 100, 'matched_state')
    # fuzzy_year = fuzzy_matches(df1_, df2_, 'fyear', 'Year', 100, 'matched_year')


    fuzzy_name = fuzzy_name[fuzzy_name['matched_name'].notnull()]
    fuzzy_add = fuzzy_add[fuzzy_add['matched_address'].notnull()]
    fuzzy_zip = fuzzy_zip[fuzzy_zip['matched_zip'].notnull()]
    fuzzy_city = fuzzy_city[fuzzy_city['matched_city'].notnull()]
    fuzzy_state = fuzzy_state[fuzzy_state['matched_state'].notnull()]
    # fuzzy_year = fuzzy_year[fuzzy_year['matched_city'].notnull()]

    # Extract the index of matched names and addresses from the tuples
    fuzzy_name['matched_name_i'] = fuzzy_name['matched_name'].apply(lambda x: x[2]  if x else None)
    fuzzy_add['matched_address_i'] = fuzzy_add['matched_address'].apply(lambda x: x[2] if x else None)
    fuzzy_zip['matched_zip_i'] = fuzzy_zip['matched_zip'].apply(lambda x: x[2] if x else None)
    fuzzy_city['matched_city_i'] = fuzzy_city['matched_city'].apply(lambda x: x[2] if x else None)
    fuzzy_state['matched_state_i'] = fuzzy_state['matched_state'].apply(lambda x: x[2] if x else None)

    intersection = pd.merge(fuzzy_name, fuzzy_add, how='inner', on=['conm', 'add1'])
    intersection = clean(intersection)
    intersection = pd.merge(intersection, fuzzy_zip, how='inner', on=['conm', 'add1', 'addzip'])
    intersection = clean(intersection)
    intersection = pd.merge(intersection, fuzzy_city, how='inner', on=['conm', 'add1', 'addzip', 'city'])
    intersection = clean(intersection)
    intersection = pd.merge(intersection, fuzzy_state, how='inner', on=['conm', 'add1', 'addzip', 'city', 'state'])
    intersection = clean(intersection)
    # intersection = pd.merge(intersection, fuzzy_year, how='inner', on=['conm', 'add1', 'addzip', 'city', 'state', 'fyear'])
    # intersection = clean(intersection)

    # Create a boolean mask where each row is True if all specified columns have the same index value
    # mask = (intersection['matched_name_i'] == intersection['matched_address_i']) & \
    #         (intersection['matched_address_i'] == intersection['matched_zip_i']) & \
    #         (intersection['matched_zip_i'] == intersection['matched_city_i']) & \
    #         (intersection['matched_city_i'] == intersection['matched_state_i'])

    ### It seems that we don't need to match attributes other than the company name and address, 
    # because the fuzzy match procedure only returns the first match, whose index is meaningless.
    mask = (intersection['matched_name_i'] == intersection['matched_address_i'])

    subset_df = intersection[mask]

    print(f'Number of fuzzy matches (Company Name & Add & Zip & City & State & Year): {len(subset_df)}')
    return intersection, subset_df


##### Method 2 test #####
inter_, sub_df = fuzzy_sep(df1_, df2_)
len(inter_)

Number of fuzzy matches (Company Name & Add & Zip & City & State & Year): 49


200

In [62]:
# intersection[intersection['matched_value'].notnull()]

### Apply on full dataset

In [32]:
# df1 = df_Compustat.copy()
# df2 = df_ODI.copy()   

In [ ]:
##### Method 1 #####
# Start the timer
start_time = time.time()

fuzzy_all = fuzzy_matches_combined(df1, df2, 90, 'matched_value')
fuzzy_all = fuzzy_all[fuzzy_all['matched_value'].notnull()]

# End the timer
end_time = time.time()
duration = end_time - start_time
print(f"The code took {duration} seconds to execute.")

fuzzy_all

In [ ]:
##### Method 2 #####
inter_all, sub_df_all = fuzzy_sep(df1, df2)
len(inter_all)

In [34]:
# # Evaluating the result of fuzzy match 
# ##### string matches #####

# # test
# # Start the timer
# start_time = time.time()

# str_matches_name = string_matches(df1, df2, 'conm', 'ESTAB_NAME')
# str_matches_add = string_matches(df1, df2, 'add1', 'STREET')
# str_matches_zip = string_matches(df1, df2, 'addzip', 'ZIP')
# str_matches_city = string_matches(df1, df2, 'city', 'CITY')
# str_matches_state = string_matches(df1, df2, 'state', 'STATE')
# str_matches_year = string_matches(df1, df2, 'fyear', 'Year')

# inter_str = pd.merge(str_matches_name, str_matches_add, how='inner', on=['conm', 'add1'])
# inter_str = clean(inter_str)
# inter_str = pd.merge(inter_str, str_matches_zip, how='inner', on=['conm', 'addzip'])
# inter_str = clean(inter_str)
# inter_str = pd.merge(inter_str, str_matches_city, how='inner', on=['conm', 'city'])
# inter_str = clean(inter_str)
# inter_str = pd.merge(inter_str, str_matches_state, how='inner', on=['conm', 'state'])
# inter_str = clean(inter_str)
# inter_str = pd.merge(inter_str, str_matches_year, how='inner', on=['conm', 'fyear'])
# inter_str = clean(inter_str)

# print(f'Number of matches (Company Name): {len(inter_str)}')

# # End the timer
# end_time = time.time()
# duration = end_time - start_time
# print(f"The code took {duration} seconds to execute.")

In [35]:
# # nested iter -> too slow #
# ## Finding results where 'conm' matches with 'ESTAB_NAME' and 'add1' matches with 'STREET' simultaneously

# def find_dual_matches(df1, df2, col1_df1, col1_df2, col2_df1, col2_df2):
#     dual_matches = []
#     for index1, row1 in df1.iterrows():
#         for index2, row2 in df2.iterrows():
#             if row1[col1_df1] == row2[col1_df2] and row1[col2_df1] == row2[col2_df2]:
#                 dual_matches.append((index1, index2))
#     return dual_matches

# # test
# dual_matches = find_dual_matches(df1, df2, 'conm', 'ESTAB_NAME', 'add1', 'STREET')
# len(dual_matches), dual_matches

## Multiprocessing

In [36]:
# def basic_fuzzy_matching(row, choices):
#     # Here you can implement a basic string comparison, for example, using startswith, endswith, or in
#     matches = [choice for choice in choices if row in choice or choice in row]
#     return matches[:3]  # Return top 3 matches

# # Function to parallelize fuzzy matching
# def parallel_fuzzy_matching(df, column_to_match, choices, num_processes=4):
#     with Pool(num_processes) as pool:
#         results = pool.map(basic_fuzzy_matching, [(row, choices) for row in df[column_to_match]])
#     return results

# matched_results = utils.parallel_fuzzy_matching(df1, 'conm', df2['ESTAB_NAME'].tolist())
# df1['matched_NAME'] = matched_resultsintersection

## Applying Additional Filters

In [69]:
# Prepare for merging
df1[['conm_', 'add1_', 'city_', 'state_']] = df1[['conm', 'add1', 'city', 'state']]
df1[['conm', 'add1', 'city', 'state']] = df1[['conm', 'add1', 'city', 'state']].apply(lambda x: x.str.lower())

df2[['ESTAB_NAME_', 'STREET_', 'CITY_', 'STATE_']] = df2[['ESTAB_NAME', 'STREET', 'CITY', 'STATE']]
df2[['ESTAB_NAME', 'STREET', 'CITY', 'STATE']] = df2[['ESTAB_NAME', 'STREET', 'CITY', 'STATE']].apply(lambda x: x.str.lower())

In [70]:
# Merge fuzzy matched subset with the original dataset
df = pd.merge(fuzzy_2, df1, how='inner', on=['conm', 'add1', 'addzip', 'city', 'state', 'fyear']) # Method 1: fuzzy_2

# Remove columns that end with '_y' and Rename columns to remove '_x'
df = df[df.columns.drop(list(df.filter(regex='_y')))]
df = df.rename(columns=lambda x: x.replace('_x', ''))

# Drop unnecessary cols
df1[['conm', 'add1', 'city', 'state']] = df1[['conm_', 'add1_', 'city_', 'state_']]
# df.drop(['conm_', 'add1_', 'city_', 'state_', 'matched_name', 'matched_address', 'matched_zip', 'matched_city', 'matched_state'], axis=1, inplace=True) # Method 2, subset_df
df.drop(['conm_', 'add1_', 'city_', 'state_', 'matched_name'], axis=1, inplace=True) # Method 1, fuzzy_2

### Splitting 'matched_value' in order to merge with df2

def split_values(row):
    if row['matched_value'] and len(row['matched_value']) > 0:
        lst = row['matched_value'][0].split(",")
        # print(lst)
        if len(lst) == 5:
            return lst[0], lst[1], lst[2], lst[3], lst[4]
    return None, None, None, None, None

# Apply the function to each row
df[['ESTAB_NAME', 'STREET', 'ZIP', 'CITY', 'STATE']] = df.apply(split_values, axis=1, result_type='expand')
# df.head()

In [71]:
# Merge with df2 and clean the final dataset

df = pd.merge(df, df2, how='inner', on=['ESTAB_NAME', 'STREET', 'ZIP', 'CITY', 'STATE'])

df[['ESTAB_NAME', 'STREET', 'CITY', 'STATE']] = df[['ESTAB_NAME_', 'STREET_', 'CITY_', 'STATE_']]
df.drop(['ESTAB_NAME_', 'STREET_', 'CITY_', 'STATE_', 'matched_value'], axis=1, inplace=True)
len(df)

114

1. Exclude financial firms and firms in regulated industries (SIC codes 6000–6999 and 4900–4999). 

In [73]:
print(f"Original DataFrame size: {len(df)}")
df_selected = df[((df['sic'] >= 4900) & (df['sic'] <= 4999)) | ((df['sic'] >= 6000) & (df['sic'] <= 6999))]
print(f"Filtered DataFrame size: {len(df_selected)}")

Original DataFrame size: 114
Filtered DataFrame size: 9


2. Limit the sample to firms that have December fiscal year ends. 

In [74]:
print(f"Original DataFrame size: {len(df_selected)}")
df_selected = df_selected[df_selected['fyr']==12]
print(f"Filtered DataFrame size: {len(df_selected)}")

Original DataFrame size: 9
Filtered DataFrame size: 0


3. Require availability of analyst forecast data from I/B/E/S

In [55]:
# Regular expression pattern to match exactly one capital letter
pattern = r'^[A-Z]$'

def check_column(column):
    return column.astype(str).str.contains(pattern, na=False).all()

# Apply the function to each column and find columns that match the criterion
matching_columns = df.apply(check_column)

matching_column_names = matching_columns[matching_columns].index.tolist()
matching_column_names

['consol', 'popsrc', 'final', 'costat', 'idbflag']

In [56]:
df[matching_column_names]

,consol,popsrc,final,costat,idbflag
0,C,D,Y,A,B
1,C,D,Y,A,B
2,C,D,Y,A,B
3,C,D,Y,A,B
4,C,D,Y,A,B
...,...,...,...,...,...
109,C,D,Y,A,D
110,C,D,Y,A,D
111,C,D,Y,A,D
112,C,D,Y,A,D


In [75]:
bag = ['I', 'B', 'E', 'S']
df_selected = df_selected[df_selected['costat'].isin(bag)] ## ? not sure which column should be 
print(f"Filtered DataFrame size: {len(df_selected)}")

Filtered DataFrame size: 0


4. Exclude Zero employees or do not include the number of employees (-254)

In [58]:
# Function to check if all values in a column are integers
def is_integer_column(column):
    if column.dtype.kind in 'iu':  # Check if column data type is integer (signed or unsigned)
        return True
    elif column.apply(lambda x: isinstance(x, int)).all():  # Check each value for integer type
        return True
    else:
        return False

# Apply the function to each column and find columns that are all integers
integer_columns = df.apply(is_integer_column)

integer_column_names = integer_columns[integer_columns].index.tolist()
integer_column_names

['fyear', 'gvkey', 'fyr', 'pddur', 'upd', 'fyrc', 'sic', 'Year']

In [59]:
df[integer_column_names]

,fyear,gvkey,fyr,pddur,upd,fyrc,sic,Year
0,2001,1602,12,12,3,12,2836,2002
1,2002,1602,12,12,3,12,2836,2002
2,2003,1602,12,12,3,12,2836,2002
3,2004,1602,12,12,3,12,2836,2002
4,2005,1602,12,12,3,12,2836,2002
...,...,...,...,...,...,...,...,...
109,2008,7343,8,12,3,8,3674,2002
110,2009,7343,8,12,3,8,3674,2002
111,2010,7343,8,12,3,8,3674,2002
112,2011,7343,8,12,3,8,3674,2002


In [76]:
print(f"Original DataFrame size: {len(df_selected)}")
df_selected = df_selected[(df_selected['EMP_Q1'] != 0) & (df_selected['EMP_Q1'].notnull())]
print(f"Filtered DataFrame size: {len(df_selected)}")

Original DataFrame size: 0
Filtered DataFrame size: 0


5. Exclude extremely low or high hours worked per employee or extremely high injury/illness rates

In [ ]:
# Eliminate observations for which hours worked per employee is at the top or bottom half-percentile 
# or the number of injury/illness cases at the top half-percentile

In [77]:
print(f"Original DataFrame size: {len(df_selected)}")

# Calculate the half-percentiles for 'HOURS_Q2'
lower_bound_hours = df_selected['HOURS_Q2'].quantile(0.005)
upper_bound_hours = df_selected['HOURS_Q2'].quantile(0.995)

# Filter out the rows
df_selected = df_selected[(df_selected['HOURS_Q2'] > lower_bound_hours) & (df_selected['HOURS_Q2'] < upper_bound_hours)]

print(f"Filtered DataFrame size: {len(df_selected)}")

Original DataFrame size: 0
Filtered DataFrame size: 0


6. we require that number of employees in the sample establishments of a given firm represents at least five percent of the total number of employees of that firm, as reported in Compustat.